In [4]:

import pandas as pd
import streamlit as st
import altair as alt
import time as time
import datetime as datetime
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
from sqlalchemy import select
import pyodbc
import openpyxl


## Conexão com banco de dados ##

# String de conexão
connection_url = URL.create(
    "mssql+pyodbc",
    username="dafonte",
    password="@d123",
    host="10.0.1.5",
    database="DBDafonte",
    query={
        "driver": "SQL Server",
        "LongAsMax": "Yes",
    },
)

## Engenharia de conexão ##
engine = create_engine(
    connection_url
    )

## Query banco ##
consulta_sql = '''

            SELECT
            DESCRMEDIDA as Serviço,
            CLIENTE as Cliente,
            PV, 
            COLETA, 
            ROW_NUMBER() OVER(PARTITION BY COLETA ORDER BY DATAENTRADAPROD ASC) AS Ordem,
            LOJA as Loja, 
            CODIGO as Cód, 
            LOCALIZACAO as Esteira,
            LEFT(DATAENTRADAPROD,3) as Mes,
            YEAR(DATAENTRADAPROD) as Ano,
            MIN(DATAENTRADAPROD) OVER(PARTITION BY COLETA) AS Entrada, 
            MAX(DATAENTRADAPROD) OVER(PARTITION BY COLETA) AS Saída,
            DATEDIFF(hour,Min(DATAENTRADAPROD) OVER(PARTITION BY COLETA),Max(DATAENTRADAPROD) OVER(PARTITION BY COLETA)) as SLA,
            LIBERACAO AS Liberacao,
            PLANOPAGT as PlanoPgto,
            PRIORIDADE as Prioridade,
            VENDEDOR as Vendedor,
            CPFCNPJ
            FROM DbDafonte.dbo.VW_KPI_VISAO_PNEU_LOCALIZACAO
            where YEAR (CONVERT(date, DATAENTRADAPROD, 101)) = YEAR(GETDATE())
            AND CLIENTE NOT LIKE '%DAFONTE - LOJA%'

              '''
## Conversão em DataFrame ##
df = pd.read_sql(sql=consulta_sql, con=engine)

In [6]:
df.head()

df.to_excel('PlanilhaProd.xlsx', sheet_name='Produção', engine='openpyxl',index=False)

In [3]:
st.download_button(
    label="Download data as CSV",
    data=csv,
    file_name='large_df.csv',
    mime='text/csv',
)

Serviço               object
Cliente               object
PV                     int64
COLETA                 int64
Ordem                  int64
Loja                   int64
Cód                    int64
Esteira               object
Mes                   object
Ano                    int64
Entrada       datetime64[ns]
Saída         datetime64[ns]
SLA                    int64
Liberacao             object
PlanoPgto             object
Prioridade            object
Vendedor              object
CPFCNPJ               object
dtype: object